# macOS Jupyter 実行基盤 + VS Code + Mermaid + アルゴリズム（総まとめ）
このノートは、今回の内容を **1つに統合** した実行可能ドキュメントです。

## 1. 環境チェック（ターミナルで実行）
```bash
which pyenv && pyenv --version
which python && pyenv which python
pyenv install -l | grep -E "^\s*3\.12\."
pyenv install 3.12.11
mkdir -p ~/projects/algorithms && cd ~/projects/algorithms
pyenv local 3.12.11
python -V
python -m venv .venv && source .venv/bin/activate
python -m pip install --upgrade pip
```


## 2. プロジェクト雛形を生成（Makefile / .vscode/settings.json）

In [ ]:

# import os
import json
import pathlib
base = pathlib.Path(".")
(base/".vscode").mkdir(parents=True, exist_ok=True)

makefile = """
# -------- Project Execution Foundation --------
PYTHON ?= python
VENV ?= .venv
KERNEL ?= algo-312
REQ ?= requirements.txt

.PHONY: ensure-venv
ensure-venv:
	@test -d $(VENV) || ($(PYTHON) -m venv $(VENV))

.PHONY: setup
setup: ensure-venv
	@. $(VENV)/bin/activate; \
	$(PYTHON) -m pip install --upgrade pip; \
	test -f $(REQ) && pip install -r $(REQ) || true; \
	$(PYTHON) -m ipykernel install --user --name $(KERNEL)

.PHONY: install
install: ensure-venv
	@. $(VENV)/bin/activate; \
	$(PYTHON) -m pip install --upgrade pip; \
	test -f $(REQ) && pip install -r $(REQ) || true
	@which bun >/dev/null 2>&1 && bun install || echo "bun not found (skip)"

.PHONY: lab
lab:
	@. $(VENV)/bin/activate; jupyter lab

.PHONY: test
test:
	@. $(VENV)/bin/activate; \
	pytest -q || echo "pytest skipped/failed (install pytest?)"
	@which bun >/dev/null 2>&1 && bunx vitest run || echo "vitest skipped (bun not found)"

.PHONY: freeze
freeze:
	@. $(VENV)/bin/activate; pip freeze --exclude-editable > requirements.lock.txt
"""

(base/"Makefile").write_text(makefile, encoding="utf-8")

settings = {
  "python.defaultInterpreterPath": "${workspaceFolder}/.venv/bin/python",
  "jupyter.askForKernelRestart": False,
  "jupyter.textOutputLimit": 100000,
  "python.analysis.typeCheckingMode": "basic"
}
(base/".vscode"/"settings.json").write_text(json.dumps(settings, indent=2), encoding="utf-8")
print("Generated -> ./Makefile, ./.vscode/settings.json")


## 3. Jupyter の最小実例（表とグラフ）

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)
df = pd.DataFrame({"x": range(1, 11), "y": np.random.randint(0, 100, 10)})
display(df)
plt.figure(figsize=(5,3))
plt.plot(df["x"], df["y"], marker="o")
plt.title("Sample Line Plot")
plt.xlabel("x")
plt.ylabel("y")
plt.tight_layout()
plt.show()


## 4. Mermaid（改行保持の安全版）

In [ ]:

from IPython.display import HTML
html = """
<div>
  <script src="https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.min.js"></script>
  <script>mermaid.initialize({ startOnLoad:false });</script>
  <pre class="mermaid" id="bs" style="white-space: pre;">
  %%{init: {'flowchart': {'htmlLabels': false, 'curve': 'linear'}}}%%
  flowchart TD
    A[Start] --> B[Compute mid]
    B --> C{A[mid] ? x}
    C -->|equal|   D[Return mid]
    C -->|less|    E[low = mid + 1]
    C -->|greater| F[high = mid - 1]
    E --> B
    F --> B
  </pre>
  <script>mermaid.run({ querySelector: '#bs' });</script>
</div>
"""
HTML(html)


## 5. SVG（静的図の生成）

In [ ]:

from IPython.display import SVG, display
svg = '<svg xmlns="http://www.w3.org/2000/svg" width="500" height="180">\n' \
    + '<defs><marker id="arrow" markerWidth="10" markerHeight="10" refX="6" refY="3" orient="auto"><path d="M0,0 L0,6 L6,3 z"/></marker></defs>' \
    + '<rect x="20" y="20" width="120" height="40" rx="6" ry="6" fill="white" stroke="black"/>' \
    + '<text x="80" y="45" text-anchor="middle" dominant-baseline="middle" font-size="12">Start</text>' \
    + '<rect x="170" y="20" width="150" height="40" rx="6" ry="6" fill="white" stroke="black"/>' \
    + '<text x="245" y="45" text-anchor="middle" dominant-baseline="middle" font-size="12">Compute mid</text>' \
    + '<polygon points="360,20 420,40 360,60 300,40" fill="white" stroke="black"/>' \
    + '<text x="360" y="40" text-anchor="middle" dominant-baseline="middle" font-size="12">A[mid] ? x</text>' \
    + '<line x1="140" y1="40" x2="170" y2="40" stroke="black" marker-end="url(#arrow)"/>' \
    + '<line x1="320" y1="40" x2="300" y2="40" stroke="black" marker-end="url(#arrow)"/>' \
    + '</svg>'
with open('binary_search_flow.svg','w') as f:
    f.write(svg)
display(SVG(svg))
print('Saved: binary_search_flow.svg')


## 6. アルゴリズム：Binary Search（実装・テスト・可視化）

In [ ]:

def binary_search(a, x):
    low, high = 0, len(a) - 1
    while low <= high:
        mid = (low + high) // 2
        if a[mid] == x:
            return mid
        if a[mid] < x:
            low = mid + 1
        else:
            high = mid - 1
    return -1

# tests
assert binary_search([], 1) == -1
assert binary_search([1], 1) == 0
assert binary_search([1,2,3,4,5], 1) == 0
assert binary_search([1,2,3,4,5], 5) == 4
assert binary_search([1,2,3,4,5], 6) == -1
print("tests: ok")


In [ ]:

import matplotlib.pyplot as plt
xs = list(range(1,21))
ys = [binary_search(xs, k) for k in xs]
plt.figure(figsize=(5,3))
plt.plot(xs, ys, marker="o")
plt.title("Index Found vs Value (Sorted Array)")
plt.xlabel("Value")
plt.ylabel("Index")
plt.tight_layout()
plt.show()
